# Football News Data Retrieval 

skysports

In [ ]:
#web scrapper class


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

In [ ]:
driver = webdriver.Chrome()

driver.get("https://www.skysports.com/") #open the web link in chrome browser


try:
     #Switch to the iframe
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "sp_message_iframe_1223386"))  # Using ID of iframe
    )
    consent_iframe = driver.find_element(By.ID, "sp_message_iframe_1223386")
    driver.switch_to.frame(consent_iframe)
    
    cookie_button = WebDriverWait(driver, 12).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="notice"]/div[5]/button[3]'))
    )
    cookie_button.click()
    #driver.execute_script("arguments[0].click();", cookie_button)
    print("Cookie popup accepted.")

     #Switch back to the main page
    driver.switch_to.default_content()
except:
    print("Failed to locate or click the cookie button.")


try:
    ''' Naviagte to -> Sports Tab -> Football tab -> News tab'''
    
    sports_button = WebDriverWait(driver,9).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="site-nav-desktop-sports-more"]')))
    sports_button.click()
    print("sports button clicked.")

    s_football_button = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="site-nav-desktop-sports-more-nav"]/div/ul/li[1]/a')))
    s_football_button.click()
    print('s_football_button clicked')

    sf_news_button = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="sdc-site-localnav-body"]/div/div/ul[1]/li[2]/a')))
    sf_news_button.click()
    print('sf_news_button clicked')

except:
    print("Failed to locate or click the required buttons")


#get all news links
#news_links = driver.find_elements(By.XPATH,'//*[@id="sdc-site-localnav-body"]/div/div/ul[1]/li[2]/a')
news_links = driver.find_elements(By.CLASS_NAME, 'sdc-site-tile__headline-link')
news_url = [ link.get_attribute('href') for link in news_links] 

news_data = []

#scrape content from each link
for url in news_url:
    
    driver.get(url) #visit article page

    try:
        #extract headline
        headline= WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH,'//h1'))).text.strip()

        #extract content
        article = driver.find_elements(By.XPATH, "//p")
        article_text = " ".join([p.text.strip() for p in article])

    #Store the data
        news_data.append({
            "title": headline,
            "url": news_url,
            "content": article_text
        })
        print(f"Scraped: {headline}")

    except Exception as e:
        print(f"Failed to scrape {news_url}: {e}")
    

#driver.quit()

Cookie popup accepted.
sports button clicked.
s_football_button clicked
sf_news_button clicked
Scraped: Man Utd transfer news: Napoli ready to offer Victor Osimhen as part of Rasmus Hojlund deal - Paper Talk
Scraped: Real Madrid 2-0 Arsenal: Ian Wright slams 'disgrace' of a pitch in Gunners' Women's Champions League defeat
Scraped: Real Madrid 2-0 Arsenal: Gunners struggle to cope with weather in Women's Champions League quarter-final first leg
Scraped: Inside Thomas Tuchel's first England training session ahead of World Cup 2026 Qualifier against Albania
Scraped: Nottingham Forest transfers, latest news, rumours and gossip: Live updates, goals and highlights
Scraped: EFL transfer news, rumours and gossip for Championship, League One and League Two
Scraped: Jean-Philippe Mateta exclusive: Crystal Palace striker reveals he wanted to play on after ear injury
Scraped: Jean-Philippe Mateta speaks on his horror incident against Millwall for the first time
Scraped: Chelsea transfer news, rum

In [30]:
print(type(news_data))
print(len(news_data))

<class 'list'>
13


In [26]:
for news in news_data:
    print("\n Title:", news['title'])
    print(news['url'])
    print("Contet Preview:", news['content'][:30], "...")


 Title: Man Utd transfer news: Napoli ready to offer Victor Osimhen as part of Rasmus Hojlund deal - Paper Talk
['https://www.skysports.com/football/transfer-paper-talk/12709/13331637/man-utd-transfer-news-napoli-ready-to-offer-victor-osimhen-as-part-of-rasmus-hojlund-deal-paper-talk', 'https://www.skysports.com/football/news/11095/13331615/real-madrid-2-0-arsenal-ian-wright-slams-disgrace-of-a-pitch-in-gunners-womens-champions-league-defeat', 'https://www.skysports.com/football/real-madrid-women-vs-arsenal-women/report/526917', 'https://www.skysports.com/football/news/12016/13331541/inside-thomas-tuchels-first-england-training-session-ahead-of-world-cup-2026-qualifier-against-albania', 'https://www.skysports.com/football/live-blog/11095/13279316/nottingham-forest-transfers-latest-news-rumours-and-gossip-live-updates-goals-and-highlights', 'https://www.skysports.com/football/live-blog/11715/13163353/efl-transfer-news-rumours-and-gossip-for-championship-league-one-and-league-two', 'htt

### *Notes:*

Challenges Encountered

1. was unable to interact with cookie permission page, when skysports page is opened.

Discoveries and Solutions

1. Checked XPath; ensure it was correct
Used Console tab in html of browsers
var element = document.evaluate('//*[@id="notice"]/div[5]/button[3]', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;console.log(element && element.offsetParent != null); returned "true"; means the path was correct

2. Adjusted script to explicitly let selenium wait for desire section to load; used method WebDriverWait
WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="notice"]/div[5]/button[3]'))

3. Examined "Elements" of the webpage; checking if "button" was located in 'iframe'. Why?
If the popup is inside an iframe, Selenium won’t see it unless you switch to the iframe

4. From inspecting the HTML under Elements tab, it was not apparent that the button was located under 'iframe' but it was observed that the title for iframe indicated 'SP COnsent Message'
The assumption is that this is related to the cookies popup permission page

4. The title="SP Consent Message" suggests it's a privacy or consent management iframe.
5. The src URL (https://cdn.privacy-mgmt.com/...) contains "privacy-mgmt", which indicates it's handling user consent.
6. the script is not clicking the button because it’s inside this iframe, and Selenium cannot interact with elements inside an iframe. Must switch to it first.



 

